In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import numpy as np
import pandas as pd
import networkx as nx
from tethne.readers import wos
import igraph
import nltk
from collections import Counter
from tethne import Corpus
from helpers import extract_keywords, filter_token, normalize_token

# 2.1. Co-citation Analysis

In this workbook we will conduct a co-citation analysis using the approach outlined in [Chen (2009)](http://arxiv.org/pdf/0904.1439.pdf). If you have used the Java-based desktop application [CiteSpace II](http://cluster.cis.drexel.edu/~cchen/citespace/), this should be familiar: this is the same methodology that is implemented in that application.

Co-citation analysis gained popularity in the 1970s as a technique for “mapping” scientific literatures, and for finding latent semantic relationships among technical publications.

Two papers are co-cited if they are both cited by the same, third, paper. The standard approach to co-citation analysis is to generate a sample of bibliographic records from a particular field by using certain keywords or journal names, and then build a co-citation graph describing relationships among their cited references. Thus the majority of papers that are represented as nodes in the co-citation graph are not papers that responded to the selection criteria used to build the dataset.

![](http://diging.github.io/tethne/doc/0.6.1-beta/_images/citationnetworks.png)

Our objective in this tutorial is to identify papers that bridge the gap between otherwise disparate areas of knowledge in the scientific literature. In this tutorial, we rely on the theoretical framework described in [Chen (2006)](http://cluster.cis.drexel.edu/~cchen/citespace/doc/jasist2006.pdf) and [Chen et al. (2009)](http://arxiv.org/pdf/0904.1439.pdf).

According to Chen, we can detect potentially transformative changes in scientific knowledge by looking for cited references that both (a) rapidly accrue citations, and (b) have high betweenness-centrality in a co-citation network. It helps if we think of each scientific paper as representing a “concept” (its core knowledge claim, perhaps), and a co-citation event as representing a proposition connecting two concepts in the knowledge-base of a scientific field. If a new paper emerges that is highly co-cited with two otherwise-distinct clusters of concepts, then that might mean that the field is adopting new concepts and propositions in a way that is structurally radical for their conceptual framework.

[Chen (2009)](http://arxiv.org/pdf/0904.1439.pdf) introduces sigma ($\Sigma$) as a metric for potentially transformative cited references:

$$
\Sigma(v) = (g(v) + 1)^{burstness(v)}
$$

...where the betweenness centrality of each node v is:

$$
g(v) = \sum\limits_{i\neq j\neq v} \frac{\sigma_{ij} (v)}{\sigma_{ij}}
$$

...where $\sigma_{ij}$ is the number of shortest paths from node i to node j and $\sigma_{ij}(v)$ is the number of those paths that pass through v. *Burstness* (0.-1. normalized) is estimated using [Kleingberg’s (2002)](http://www.cs.cornell.edu/home/kleinber/bhs.pdf) automaton model, and is designed to detect rate-spikes around features in a stream of documents.

----- ------
**Note**: In this notebook we will not use burstness, but rather the relative increase/decrease in citations from one year to the next. Burstness is helpful when we are dealing with higher-resolution time-frames, and/or we want to monitor a long stream of citation data. Since we will smooth our data with a multi-year time-window, burstness becomes a bit less informative, and the year-over-year change in citations (we'll call this Delta $\Delta$) is an intuitive alternative.
----- ------

## Load data

Here we have some field-tagged data from the Web of Science. We set ``streaming=True`` so that we don't load everything into memory all at once. 

--- ---
**Note**:  When we stream the corpus, it is important to set ``index_fields`` and ``index_features`` ahead of time, so that we don't have to iterate over the whole corpus later on.
--- ---

In [43]:
metadata = wos.read('../data/Baldwin/PlantPhysiology', 
                    streaming=True, index_fields=['date', 'abstract'], index_features=['citations'])

In [44]:
len(metadata)

7849

## Co-citation graph

``Tethne`` provides a function called ``cocitation`` that creates co-citation graphs.

In [5]:
from tethne import cocitation

Co-citation graphs can get enormous quickly, and so it is important to set a threshold number of times that a paper must be cited to be included in the graph (``min_weight``). It's better to start high, and bring the threshold down as needed.

Note that ``edge_attrs`` should be set to whatever was the value of ``index_fields`` when we used ``read()`` (above).

In [6]:
graph = cocitation(metadata, min_weight=6., edge_attrs=['date'])

In [7]:
graph.order(), graph.size(), nx.number_connected_components(graph)

(6537, 23121, 206)

### Serialize

We can visualize our graph in Cytoscape to get a sense of its structure.

In [8]:
nx.write_graphml(graph, 'cocitation.graphml')

## Sigma, $\Sigma$

Chen (2009) proposed sigma ($\Sigma$) as a metric for potentially transformations in a scientific literature.

$$
\Sigma(v) = (g(v) + 1)^{burstness(v)}
$$

----- ------
**Note**: In this notebook we will not use burstness, but rather the relative increase/decrease in citations from one year to the next. Burstness is helpful when we are dealing with higher-resolution time-frames, and/or we want to monitor a long stream of citation data. Since we will smooth our data with a multi-year time-window, burstness becomes a bit less informative, and the year-over-year change in citations (we'll call this Delta $\Delta(v)$) is an intuitive alternative. So:

$$
\Sigma(v) = (g(v) + 1)^{\Delta(v)}
$$

$$
\Delta(v) = \frac{N_t(v) - N_{t-1} }{max(1, N_{t-1})}
$$
----- -----

## GraphCollection

Since we are interested in the evolution of the co-citation graph over time, we need to create a series of sequential graphs. Tethne provides a class called ``GraphCollection`` that will do this for us.

We pass ``metadata`` (or ``Corpus`` object), the ``cotation`` function, and then some configuration information:
* ``slice_kwargs`` controls how the sequential time-slices are generated. The default is to use 1-year slices, and advance 1 year per slice. We are stating here that we want to extract only the ``citations`` feature from each slice (for performance).
* ``method_kwargs`` controls the graph-building function. Here we pass our ``min_weight``, and we also say that we don't want any attributes on the edges (for performance).

In [9]:
from tethne import GraphCollection
G = GraphCollection(metadata, cocitation,
                    slice_kwargs={'feature_name': 'citations'}, 
                    method_kwargs={'min_weight': 3, 'edge_attrs': []})

In [12]:
for year, graph in G.iteritems():
    print graph.order(), graph.size(), nx.number_connected_components(graph)
    nx.write_graphml(graph, 'cocitation_%i.graphml' % year)

406 1019 57
516 1257 60
682 1536 77
626 1336 67
917 2144 82
834 2241 91
627 1154 97
609 1657 65
660 1515 79
868 2215 68
792 1818 86
774 1657 83
839 2227 81
845 2549 71
391 638 67


## Betweenness centrality

Recall that the betweenness centrality of each node v is:

$$
g(v) = \sum\limits_{i\neq j\neq v} \frac{\sigma_{ij} (v)}{\sigma_{ij}}
$$

We can analyze all of the nodes in all of the graphs in our ``GraphCollection`` using its ``analyze()`` method.

In [13]:
# 'betweenness_centrality' is the name of the algorithm
#  in NetworkX that we want to use. ``invert=True`` means
#  that we want to organize the g(v) values by node, rather
#  than by time-period (the default).
g_v = G.analyze('betweenness_centrality', invert=True)

In [26]:
g_v.items()[89]

(89,
 {1999: 0.012212559880326648,
  2000: 0.1451754690517866,
  2001: 0.11777211863697992,
  2002: 0.024040269161038387,
  2003: 0.012573209158147235,
  2004: 0.05753114988769691,
  2005: 0.002120708428951243,
  2006: 0.026932066769497706,
  2007: 0.025856622517020283,
  2008: 0.03452387454835741,
  2009: 0.011951828484182447,
  2010: 0.00529130551685833,
  2011: 0.008302012907130655,
  2012: 0.00036514767024372035,
  2013: 4.394348867356579e-06})

### Organize our data

In order to calculate $\Sigma$ more efficiently, we'll organize our data about the graph in a DataFrame. Our DataFrame will have the following columns:

* ID - The GraphCollection gives each node an integer ID. We'll need this to identify nodes in Cytoscape, later.
* Node - This is the Author Year Journal label from the WoS data for a cited reference.
* Citations - The number of citations.
* Centrality - Betweenness centrality.
* Delta - The year-over-year increase/decrease in citations.

This may take a bit, depending on the size of the graphs.

In [23]:
node_data = pd.DataFrame(columns=['ID', 'Node', 'Year', 'Citations', 'Centrality', 'Delta'])

i = 0
for n in G.node_index.keys():
    if n < 0:
        continue
        
    # node_history() gets the values of a node attribute over
    #  all of te graphs in the GraphCollection.
    g_n = G.node_history(n, 'betweenness_centrality')
    N_n = G.node_history(n, 'count')
    
    # Skip nodes whose g(v) never gets above 0.
    if max(g_n.values()) == 0:
        continue
    
    years = sorted(G.keys())   # Graphs are keyed by year.
    for year in years:    
        g_nt = g_n.get(year, 0.0)    # Centrality for this year.
        N_nt = float(N_n.get(year, 0.0))    # Citations for this year.
        # For the second year and beyond, calculate Delta.
        if year > years[0]:    
            N_nlast = N_n.get(year-1, 0.0)
            delta = (N_nt - N_nlast)/max(N_nlast, 1.)
        else:    
            delta = 0.0
            
        # We will add one row per node per year.
        node_data.loc[i] = [n, G.node_index[n], year, N_nt, g_nt, delta]
        i += 1

That was fairly computationally expensive. We should save the results so that we don't have to do that again.

In [29]:
node_data.to_csv('node_data.csv', encoding='utf-8')

Before calculating $\Sigma$ we will select a subset of the rows in our dataframe, to reduce the computational burden. Here we create a smaller DataFrame with only those rows in which both Centrality and Delta are not zero -- it should be obvious that these will have negligible $\Sigma$.



In [30]:
# Note the ``.copy()`` at the end -- this means that the new DataFrame will be a
#  stand-alone copy, and not just a "view" of the existing ``node_data`` DataFrame.
#  The practical effect is that we can add new data to the new ``candidates``
#  DataFrame without creating problems in the larger ``node_data`` DataFrame.
candidates = node_data[node_data.Centrality*node_data.Delta > 0.].copy()

Now we calculate $\Sigma$. Vector math is great!

In [31]:
candidates['Sigma'] = (1.+candidates.Centrality)**candidates.Delta

The nodes (in a given year) with the highest $\Sigma$ are our candidates for potential "transformations". Note that in Chen's model, the cited reference and its co-cited references are an emission of the "knowledge" of the field. In other words, the nodes and edges in our graph primarily say something about the records that are doing the **citing**, rather than the records that are **cited**. I.e. a node with $\Sigma$ is *indicative* of a transformation, but that is a description of the papers that cite it and not necessarily the paper that the node represents.

In [32]:
candidates.sort('Sigma', ascending=False)

,ID,Node,Year,Citations,Centrality,Delta,Sigma
29141,5060,BROWN_DM_2005_PLANT_CELL,2010,18,1.648243e-01,18.000000,15.584316
10849,1768,EISEN_MB_1998_P_NATL_ACAD_SCI_USA,2003,12,1.404663e-01,12.000000,4.841608
13774,2284,ULMASOV_T_1997_PLANT_CELL,2003,11,1.216092e-01,11.000000,3.533924
5792,895,KAUL_S_2000_NATURE,2001,32,3.894427e-02,32.000000,3.395878
33433,6039,PIETERSE_CMJ_2009_NAT_CHEM_BIOL,2012,15,4.728827e-02,15.000000,1.999832
2761,411,BUSH_DS_1995_ANNU_REV_PLANT_PHYS,2000,10,6.533943e-02,10.000000,1.883129
25764,4413,RUSSINOVA_E_2004_PLANT_CELL,2008,9,7.181309e-02,9.000000,1.866687
20094,3441,BENJAMINI_Y_1995_J_ROY_STAT_SOC_B_MET,2008,19,3.334857e-02,19.000000,1.865048
14855,2476,SHI_HZ_2002_PLANT_CELL,2004,9,6.633559e-02,9.000000,1.782564
16132,2728,KWAK_JM_2003_EMBO_J,2006,11,5.320022e-02,11.000000,1.768562


## Cluster labeling

In [24]:
clusters = pd.read_csv('clusters.dat', sep='\t', skiprows=9)

In [25]:
clusters

,Cluster,Score (Density*#Nodes),Nodes,Edges,Node IDs
0,1,22.083,25,265,"FUJIOKA_S_1997_NAT_PROD_REP, NOMURA_T_1997_PLA..."
1,2,13.333,16,100,"AUKERMAN_MJ_1997_PLANT_CELL, QUAIL_PH_1995_SCI..."
2,3,12.889,19,116,"BASU_U_1994_J_PLANT_PHYSIOL, SIVAGURU_M_1998_P..."
3,4,12.615,14,82,"KIEBER_JJ_1993_CELL, WILKINSON_JQ_1997_NAT_BIO..."
4,5,10.400,11,52,"MCQUEENMASON_S_1994_P_NATL_ACAD_SCI_USA, BRUMM..."
5,6,10.000,10,45,"GRIFFITH_M_1992_PLANT_PHYSIOL, KROL_M_1984_CAN..."
6,7,9.800,11,49,"SENTENAC_H_1992_SCIENCE, SCHACHTMAN_DP_1994_NA..."
7,8,9.400,11,47,"NI_BR_1993_PLANT_PHYSIOL, GROOT_SPC_1992_PLANT..."
8,9,9.111,10,41,"PERRY_SE_1994_PLANT_CELL, HIRSCH_S_1994_SCIENC..."
9,10,9.000,9,36,"POTIKHA_TS_1999_PLANT_PHYSIOL, WINGE_P_1997_PL..."


In [33]:
clusters['Node IDs'][0].split(', ')

['FUJIOKA_S_1997_NAT_PROD_REP',
 'NOMURA_T_1997_PLANT_PHYSIOL',
 'FUJIOKA_S_1997_PLANT_CELL',
 'CLOUSE_SD_1999_BRASSINOSTEROIDS',
 'SZEKERES_M_1996_CELL',
 'CHOI_YH_1997_PHYTOCHEMISTRY',
 'KAUSCHMANN_A_1996_PLANT_J',
 'AZPIROZ_R_1998_PLANT_CELL',
 'BISHOP_GJ_1996_PLANT_CELL',
 'LI_JM_1997_CELL',
 'NOGUCHI_T_1999_PLANT_PHYSIOL',
 'YOKOTA_T_1997_TRENDS_PLANT_SCI',
 'FUJIOKA_S_1997_PHYSIOL_PLANTARUM',
 'CHOE_SW_1999_PLANT_CELL',
 'CHOE_S_1999_PLANT_PHYSIOL',
 'FUJIOKA_S_1996_PLANT_CELL_PHYSIOL',
 'BISHOP_GJ_1999_P_NATL_ACAD_SCI_USA',
 'LI_JM_1996_SCIENCE',
 'KLAHRE_U_1998_PLANT_CELL',
 'CLOUSE_SD_1998_ANNU_REV_PLANT_PHYS',
 'CHOE_SW_1998_PLANT_CELL',
 'FUJIOKA_S_1995_BIOSCI_BIOTECH_BIOCH',
 'NOMURA_T_1999_PLANT_PHYSIOL',
 'TAKAHASHI_T_1995_GENE_DEV',
 'CLOUSE_SD_1996_PLANT_PHYSIOL']

In [34]:
citations = metadata.features['citations']

In [35]:
citing = Counter()
for reference in clusters['Node IDs'][0].split(', '):
    for idx in citations.papers_containing(reference):
        citing[idx] += 1.
chunk = [idx for idx, value in citing.items() if value > 2.]        

This step can take a few minutes.

In [48]:
abstracts = {}
for abstract, wosid in metadata.indices['abstract'].iteritems():
    print '\r', wosid[0],
    abstracts[wosid[0]] = abstract


WOS:000185974800027


In [49]:
abstracts.items()[5]

(u'WOS:000283710300030',
 u'Actinorhizal symbioses are mutualistic interactions between plants and the soil bacteria Frankia that lead to the formation of nitrogen-fixing root nodules. Little is known about the signaling mechanisms controlling the different steps of the establishment of the symbiosis. The plant hormone auxin has been suggested to play a role. Here we report that auxin accumulates within Frankia-infected cells in actinorhizal nodules of Casuarina glauca. Using a combination of computational modeling and experimental approaches, we establish that this localized auxin accumulation is driven by the cell-specific expression of auxin transporters and by Frankia auxin biosynthesis in planta. Our results indicate that the plant actively restricts auxin accumulation to Frankia-infected cells during the symbiotic interaction.')

In [85]:
document_token_counts = nltk.ConditionalFreqDist([
    (wosid, normalize_token(token))
    for wosid, abstract in abstracts.items()
    for token in nltk.word_tokenize(abstract)
    if filter_token(token)
])

In [91]:
extract_keywords(document_token_counts, lambda k: k in chunk)

array([u'br', u'brassinosteroid', u'dwarf', u'brassinolide',
       u'brassinazole', u'dpy', u'ikb', u'campestanol', u'campesterol',
       u'tracheary', u'biosynthetic', u'characterization', u'five',
       u'deficient', u'ser', u'late', u'castasterone', u'ika', u'sterol',
       u'dwarfism'], 
      dtype='<U19')

In [93]:
cluster_keywords = {}
for i, row in clusters.iterrows():
    citing = Counter()
    for reference in row['Node IDs'].split(', '):
        for idx in citations.papers_containing(reference):
            citing[idx] += 1.
    chunk = [idx for idx, value in citing.items() if value > 2.]     
    
    cluster_keywords[row.Cluster] = extract_keywords(document_token_counts, lambda k: k in chunk)

In [94]:
cluster_keywords

{1: array([u'br', u'brassinosteroid', u'dwarf', u'brassinolide',
        u'brassinazole', u'dpy', u'ikb', u'campestanol', u'campesterol',
        u'tracheary', u'biosynthetic', u'characterization', u'five',
        u'deficient', u'ser', u'late', u'castasterone', u'ika', u'sterol',
        u'dwarfism'], 
       dtype='<U19'),
 2: array([u'phya', u'phyb', u'phytochrome', u'light', u'irradiation',
        u'pulse', u'bvr', u'flowering', u'destruction', u'phytochromobilin',
        u'response', u'lfy', u'intermittent', u'shade', u'swelling',
        u'eafl', u'hir', u'red', u'camv', u'peeled'], 
       dtype='<U19'),
 3: array([u'dtz', u'citrate', u'efflux', u'malate', u'root', u'anion',
        u'aluminum', u'organic', u'callose', u'across', u'secretion',
        u'var', u'signalgrass', u'border', u'peroxidation', u'exudation',
        u'triticale', u'rootlet', u'solution', u'ruzigrass'], 
       dtype='<U19'),
 4: array([u'ethylene', u'receptor', u'ac', u'epi', u'ostmk', u'adh',
        